In [8]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# ===========================
# CONFIGURATION (your mappings)
# ===========================
CONFIG = {
    "join_key_actions": "CategoryID",
    "join_key_categories": "CategoryID",

    "action_id": "ActionID",
    "action_title": "Action",            # short label / title
    "action_description": "Explanation", # detailed explanation
    "category_name": "Category"          # sentiment category name
}

# Output files
OUT_INTERVENTIONS = "sentiment_interventions.csv"
OUT_ROI = "sentiment_roi_reference.csv"

# ===========================
# 1) Load source files
# ===========================
actions = pd.read_csv("actions_3NF.csv")
categories = pd.read_csv("categories_3NF.csv")

# Normalize column names (remove spaces, lower)
def normalize_cols(df):
    df = df.copy()
    df.columns = df.columns.str.strip().str.replace(r"\s+", "", regex=True)
    return df

actions = normalize_cols(actions)
categories = normalize_cols(categories)

# ===========================
# 2) Merge actions with categories
# ===========================
merged = actions.merge(
    categories[[CONFIG["join_key_categories"], CONFIG["category_name"]]],
    left_on=CONFIG["join_key_actions"],
    right_on=CONFIG["join_key_categories"],
    how="left"
)

if merged[CONFIG["category_name"]].isna().all():
    raise ValueError("Merge worked but all category names are NaN. Check join keys or names in CONFIG.")

# ===========================
# 3) ROI logic (based on category keywords)
# ===========================
def roi_range(cat):
    if pd.isna(cat):
        return (5, 10)
    cat = str(cat).lower()
    if "trust" in cat or "transpar" in cat:
        return (4, 9)   # harder to rebuild, slower gains
    if "engage" in cat or "motivation" in cat or "recognition" in cat:
        return (7, 14)  # strong short-term morale boost
    if "leader" in cat or "vision" in cat or "communication" in cat:
        return (8, 16)  # strategic, top-down impact
    if "adapt" in cat or "change" in cat or "resilience" in cat:
        return (6, 12)  # modest but sustainable improvement
    return (6, 12)

low_high = merged[CONFIG["category_name"]].map(roi_range)
merged["roi_min"], merged["roi_max"] = [lo for lo, hi in low_high], [hi for lo, hi in low_high]

# ===========================
# 4) Build sentiment_interventions.csv
# ===========================
interventions = pd.DataFrame({
    "intervention_id": merged[CONFIG["action_id"]],
    "intervention_name": merged[CONFIG["action_title"]],
    "intervention_description": merged[CONFIG["action_description"]].fillna("No detailed description provided."),
    "target_area": "Sentiment",
    "sentiment_category": merged[CONFIG["category_name"]],
    "roi_min_percent": merged["roi_min"],
    "roi_max_percent": merged["roi_max"]
}).drop_duplicates(subset=["intervention_id"]).reset_index(drop=True)

interventions.to_csv(OUT_INTERVENTIONS, index=False)
print(f"Created {OUT_INTERVENTIONS} with {len(interventions)} rows")

# ===========================
# 5) Build sentiment_roi_reference.csv
# ===========================
roi = interventions[[
    "intervention_id",
    "sentiment_category",
    "roi_min_percent",
    "roi_max_percent"
]].copy()

roi["metric"] = roi["sentiment_category"].astype(str)
roi["roi_description"] = "Estimated improvement in " + roi["metric"].str.lower() + " (%)"

roi = roi.rename(columns={
    "roi_min_percent": "improvement_min_percent",
    "roi_max_percent": "improvement_max_percent"
})[[
    "intervention_id",
    "metric",
    "roi_description",
    "improvement_min_percent",
    "improvement_max_percent"
]].drop_duplicates(subset=["intervention_id"])

roi.to_csv(OUT_ROI, index=False)
print(f"Created {OUT_ROI} with {len(roi)} rows")

# ===========================
# 6) Preview
# ===========================
print("\nPreview: sentiment_interventions.csv")
display(interventions.head(10))

print("\nPreview: sentiment_roi_reference.csv")
display(roi.head(10))

Created sentiment_interventions.csv with 75 rows
Created sentiment_roi_reference.csv with 75 rows

Preview: sentiment_interventions.csv


,intervention_id,intervention_name,intervention_description,target_area,sentiment_category,roi_min_percent,roi_max_percent
0,1,Hold monthly 'Autonomy Reviews' where teams fl...,This procedural approach validates user frustr...,Sentiment,The Intrusive AI,6,12
1,2,Create a 'Too Helpful AI' meme board where emp...,Playfully calling out AI overreach helps norma...,Sentiment,The Intrusive AI,6,12
2,3,Run a biweekly 'Who’s in Charge?' reflection w...,"By celebrating small acts of human override, t...",Sentiment,The Intrusive AI,6,12
3,4,Launch quarterly 'Fit Check Forums' where empl...,This structured process acknowledges irritatio...,Sentiment,The Unadaptive AI,6,12
4,5,Host a 'Mismatch Bingo' game where staff fill ...,Playful exaggeration turns repeated AI misfits...,Sentiment,The Unadaptive AI,6,12
5,6,Introduce a 'One Change Challenge' where each ...,This reflective activity validates the ingenui...,Sentiment,The Unadaptive AI,6,12
6,7,Run 'Empathy Clinics' where employees share sh...,This formal process validates subtle emotional...,Sentiment,The Uncaring AI,6,12
7,8,Start a 'Flat Reply Hall of Fame' where staff ...,Playfully spotlighting uncaring AI outputs red...,Sentiment,The Uncaring AI,6,12
8,9,"Introduce 'Warmth Tokens'—each week, employees...","By rewarding small acts of adding empathy, thi...",Sentiment,The Uncaring AI,6,12
9,10,Hold weekly ‘Decision Story Circles’ where one...,"By converting outputs into shared reasoning, t...",Sentiment,The Confusing AI,6,12



Preview: sentiment_roi_reference.csv


,intervention_id,metric,roi_description,improvement_min_percent,improvement_max_percent
0,1,The Intrusive AI,Estimated improvement in the intrusive ai (%),6,12
1,2,The Intrusive AI,Estimated improvement in the intrusive ai (%),6,12
2,3,The Intrusive AI,Estimated improvement in the intrusive ai (%),6,12
3,4,The Unadaptive AI,Estimated improvement in the unadaptive ai (%),6,12
4,5,The Unadaptive AI,Estimated improvement in the unadaptive ai (%),6,12
5,6,The Unadaptive AI,Estimated improvement in the unadaptive ai (%),6,12
6,7,The Uncaring AI,Estimated improvement in the uncaring ai (%),6,12
7,8,The Uncaring AI,Estimated improvement in the uncaring ai (%),6,12
8,9,The Uncaring AI,Estimated improvement in the uncaring ai (%),6,12
9,10,The Confusing AI,Estimated improvement in the confusing ai (%),6,12


In [9]:
import pandas as pd
import numpy as np

# --- 1. Define synthetic capability interventions ---
data = [
    {
        "intervention_id": 101,
        "intervention_title": "Define AI Strategy Roadmap",
        "intervention_description": "Develop and communicate a clear enterprise-wide AI strategy aligned with business goals.",
        "target_area": "Capability",
        "capability_dimension": "Strategy and Vision",
        "roi_min_percent": 8,
        "roi_max_percent": 14
    },
    {
        "intervention_id": 102,
        "intervention_title": "Improve Data Governance & Quality",
        "intervention_description": "Establish data standards, ownership, and governance to enhance data reliability and accessibility.",
        "target_area": "Capability",
        "capability_dimension": "Data",
        "roi_min_percent": 7,
        "roi_max_percent": 12
    },
    {
        "intervention_id": 103,
        "intervention_title": "Modernize Technology Stack",
        "intervention_description": "Upgrade legacy systems and integrate AI-ready infrastructure for scalability and interoperability.",
        "target_area": "Capability",
        "capability_dimension": "Technology",
        "roi_min_percent": 6,
        "roi_max_percent": 10
    },
    {
        "intervention_id": 104,
        "intervention_title": "Upskill Workforce in AI Literacy",
        "intervention_description": "Deliver targeted training and mentorship programs to build AI capabilities across roles and levels.",
        "target_area": "Capability",
        "capability_dimension": "Talent and Skills",
        "roi_min_percent": 10,
        "roi_max_percent": 18
    },
    {
        "intervention_id": 105,
        "intervention_title": "Embed AI into Core Processes",
        "intervention_description": "Redesign workflows and decision-making processes to leverage AI-driven insights and automation.",
        "target_area": "Capability",
        "capability_dimension": "Organisation and Processes",
        "roi_min_percent": 9,
        "roi_max_percent": 15
    },
    {
        "intervention_id": 106,
        "intervention_title": "Launch AI Innovation Lab",
        "intervention_description": "Create a cross-functional innovation hub to experiment and pilot emerging AI solutions.",
        "target_area": "Capability",
        "capability_dimension": "Innovation",
        "roi_min_percent": 8,
        "roi_max_percent": 16
    },
    {
        "intervention_id": 107,
        "intervention_title": "Strengthen Change Management for AI",
        "intervention_description": "Implement structured change management to accelerate adoption and minimize resistance.",
        "target_area": "Capability",
        "capability_dimension": "Adaptation & Adoption",
        "roi_min_percent": 6,
        "roi_max_percent": 11
    },
    {
        "intervention_id": 108,
        "intervention_title": "Implement Responsible AI Framework",
        "intervention_description": "Develop ethical AI principles and governance to ensure transparency, fairness, and accountability.",
        "target_area": "Capability",
        "capability_dimension": "Ethics and Responsibility",
        "roi_min_percent": 5,
        "roi_max_percent": 9
    }
]

cap_interventions = pd.DataFrame(data)

# --- 2. Create ROI reference table ---
roi = cap_interventions[[
    "intervention_id",
    "capability_dimension",
    "roi_min_percent",
    "roi_max_percent"
]].copy()

roi["metric"] = roi["capability_dimension"]
roi["roi_description"] = "Estimated improvement in " + roi["metric"].str.lower() + " capability (%)"

roi = roi.rename(columns={
    "roi_min_percent": "improvement_min_percent",
    "roi_max_percent": "improvement_max_percent"
})[[
    "intervention_id",
    "metric",
    "roi_description",
    "improvement_min_percent",
    "improvement_max_percent"
]]

# --- 3. Save outputs ---
cap_interventions.to_csv("synthetic_capability_interventions.csv", index=False)
roi.to_csv("synthetic_capability_roi_reference.csv", index=False)

display(cap_interventions.head())
display(roi.head())

,intervention_id,intervention_title,intervention_description,target_area,capability_dimension,roi_min_percent,roi_max_percent
0,101,Define AI Strategy Roadmap,Develop and communicate a clear enterprise-wid...,Capability,Strategy and Vision,8,14
1,102,Improve Data Governance & Quality,"Establish data standards, ownership, and gover...",Capability,Data,7,12
2,103,Modernize Technology Stack,Upgrade legacy systems and integrate AI-ready ...,Capability,Technology,6,10
3,104,Upskill Workforce in AI Literacy,Deliver targeted training and mentorship progr...,Capability,Talent and Skills,10,18
4,105,Embed AI into Core Processes,Redesign workflows and decision-making process...,Capability,Organisation and Processes,9,15


,intervention_id,metric,roi_description,improvement_min_percent,improvement_max_percent
0,101,Strategy and Vision,Estimated improvement in strategy and vision c...,8,14
1,102,Data,Estimated improvement in data capability (%),7,12
2,103,Technology,Estimated improvement in technology capability...,6,10
3,104,Talent and Skills,Estimated improvement in talent and skills cap...,10,18
4,105,Organisation and Processes,Estimated improvement in organisation and proc...,9,15
